# Script that plots and extract statistics for the diagnoses tags

### Import libs

In [1]:
import pandas as pd
import json
import os
import cv2
from Policies import *
import shutil
from tqdm import tqdm

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

#os.environ["GOTO_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=4
#os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=4 
#os.environ["OMP_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=6

### Root to directories

In [2]:
# Annotation directory
_ann_json_dir = "/home/franko/Desktop/Frlan/Novo/Data/supervisely/wrist/ann"
# Image directiory 
_img_dir = "/home/franko/Desktop/Frlan/Novo/Data/supervisely/wrist/img"

# Set files
_img_dir_filtered = "/home/franko/Desktop/Frlan/Novo/Data/filtered/img"
_ann_json_dir_filtered = "/home/franko/Desktop/Frlan/Novo/Data/filtered/ann"

### Read data and extract information

In [3]:
# COUNT EVERYTHING!!!! :)
# Function that extract data
_diagnoses_list, _, _ = obtain_data(_ann_json_dir)

# Obtain dictionary
_diagnoses_dict = build_dictionary_from_list(_diagnoses_list)
_diagnoses_dict = OrderedDict(sorted(_diagnoses_dict.items(), key=lambda x: x[1], reverse=True))
print(_diagnoses_dict)


OrderedDict([('23r-M/3.1', 5172), ('23r-M/2.1', 3627), ('23u-E/7', 3212), ('23u-M/2.1', 2330), ('23-M/3.1', 1667), ('23r-E/2.1', 1433), ('23-M/2.1', 1410), ('22r-D/2.1', 162), ('23r-E/1', 106), ('22-D/2.1', 88), ('22r-D/4.1', 81), ('23u-M/3.1', 78), ('23u-E/2.1', 78), ('23u-E/1', 66), ('22u-D/2.1', 62), ('72B(b)', 52), ('72B(c)', 38), ('22-D/4.1', 31), ('23-E/2.1', 30), ('22u-D/1.1', 26), ('22u-D/4.1', 24), ('23r-E/7', 24), ('23u-E/3', 22), ('23r-E/3', 18), ('22r-D/5.1', 14), ('23u-E/1.1', 10), ('22r-D/1.1', 10), ('72B.(b)', 10), ('22-D/1.1', 10), ('23r-E/4.1', 9), ('23u-E7', 8), ('23r-E/4.2', 8), ('23r-E/2.2', 6), ('22r-D/1', 6), ('23u-E/4', 5), ('23u-M/2.', 5), ('23r-D/2.1', 4), ('22r-D/3.1', 4), ('77.5.1A', 4), ('23r-M3.1', 4), ('23r-M2.1', 4), ('23u/E/7', 4), ('23u-M2.1', 3), ('77.1.1A', 3), ('23r-E/3.1', 2), ('23r-/M/2.1', 2), ('23-E/7', 2), ('77.2.1A', 2), ('77.3.1C', 2), ('77.4.1A', 2), ('22r-M/2.1', 2), ('22-D/5.1', 2), ('23r-D/3.1', 2), ('23-M2.1', 2), ('23-E/1', 2), ('23u-D/2

### Filter fractures --> remove all fractures with fracture missing or less than 20 fractures count


In [4]:
# Filter data to new file which contains only selected fractures
# Fractures types list
_fractures_types_list = []
for _key, _value in _diagnoses_dict.items():
    if _value >= 20:
        _fractures_types_list.append(_key)
# Add typo...
_fractures_types_list.append('72B.(b)')

# Set files
_img_dir_filtered = "/home/franko/Desktop/Frlan/Novo/Data/filtered/img"
_ann_json_dir_filtered = "/home/franko/Desktop/Frlan/Novo/Data/filtered/ann"

# Read files
_ann_files = os.listdir(_ann_json_dir)

# Split
# Progress bar
_bar = tqdm(total = len(_ann_files), leave = False)
_counter = 0
for _i, _file_name in enumerate(_ann_files):
    # Progress
    _bar.update()
    
    # Calculate paths
    _name = _file_name.split('.')[0] 
    _image_path = _img_dir+"/"+_name+".png"
    _ann_json_path = _ann_json_dir+"/"+_name+".json"
    
     # Open json dir
    _f = open(_ann_json_path)
    _data = json.load(_f)
    
     # Grab tags and objects
    _tags = _data['tags']

    # Check if tag type is dictionary --> consis diagnosis
    _fracture_types = []
    _tag = _tags[0]
    if type(_tag) is not dict:
        # Skip if no tag == no fracture
        _counter+=1
        continue
    else:       
        if len(_tag['value'].split(",")) > 1:
            _fracture_types += _tag['value'].split(",")
        else:
            _fracture_types += _tag['value'].split(";")
    # Remove spaces
    _fracture_types = ["".join(_x.split()) for _x in _fracture_types]
    
    # Export
    if set(_fracture_types).issubset(set(_fractures_types_list)):
        shutil.copy(_image_path, _img_dir_filtered+"/"+_name+".png")
        shutil.copy(_ann_json_path, _ann_json_dir_filtered+"/"+_name+".json")
    else:
        _counter+=1;
_bar.close()    
print(_counter)

6332


### Extract data --> split laterals to separate file

### Check scenarios

In [7]:
# Read files
_ann_files = os.listdir(_ann_json_dir_filtered)

# Progress bar
_bar = tqdm(total = len(_ann_files), leave = False)
_main_output_dir = {}
# Go trough files
for _file_name in _ann_files:
    # Progress
    _bar.update()
    
    # Get name
    _name = _file_name.split('.')[0] 

    # Get data
    _data = get_data_for_fracture_case(_name, _img_dir_filtered, _ann_json_dir_filtered)
    # Get scenario
    #_scenario, _name = check_scenarios(_data, "/home/franko/Desktop/Frlan/Novo/Scripts/Results")
    _scenario, _name = check_scenarios(_data, "/media/franko/930d77c2-8105-4299-a2be-4876bd39ac36/RetrivalData/Results")
    #_scenario, _name = check_scenarios(_data)
    _image, _fractures_positions, _fracture_types, _side, _projection, _original_name = _data
    
    
    if _scenario in [5.15, 4.25, 4.15, 3.15, -1, 0, 1.15]:
        print("ELLO")
        for _fracture in _fracture_types:
            if _fracture in _main_output_dir:
                _main_output_dir[_fracture] += 1
            else:
                _main_output_dir[_fracture] = 1

    
    # Here you can extract some patricular files if want
    #if _projection == 'lat' and (_scenario == 3.2 or _scenario == 4.2 or _scenario == 5.1):
        #if len(_fractures_positions) == 3 and _scenario not in[0, -1, 3.15, 4.25, 4.15, 4.25, 5.15]:
        #_file = "/home/franko/Desktop/Frlan/Novo/Scripts/Results/"+_name+"/Original_images/marked.png"
        #    _file = "/media/franko/930d77c2-8105-4299-a2be-4876bd39ac36/BLAH/ResultsT/"+_name+"/Original_images/marked.png"
        #_file2 = "/home/franko/Desktop/Frlan/Novo/Scripts/CheckResults/"+_original_name+".png"
            #_file2 = "/media/franko/930d77c2-8105-4299-a2be-4876bd39ac36/BLAH/CheckT/"+_original_name+".png"
        #    shutil.copy(_file, _file2)
_bar.close()    
    


  0%|                                         | 3/13995 [00:00<07:57, 29.28it/s]

SCENARIO 1: COUNT == POSITIONS == 1: 2156_1053129089_01_WRI-R2_M013
SCENARIO 4: COUNT < POSITIONS: 5124_0730677110_01_WRI-L1_M009
	 SCENARIO 4.1: 5124_0730677110_01_WRI-L1_M009
SCENARIO 4: COUNT < POSITIONS: 3874_0727215814_01_WRI-L1_M006
	 SCENARIO 4.1: 3874_0727215814_01_WRI-L1_M006
SCENARIO 4: COUNT < POSITIONS: 1510_1081081183_03_WRI-L1_M006
	 SCENARIO 4.1: 1510_1081081183_03_WRI-L1_M006


  0%|                                         | 6/13995 [00:00<08:19, 27.99it/s]

SCENARIO 2: COUNT > POSITIONS: 4562_0499940005_02_WRI-R2_M003
ELLO
SCENARIO 1: COUNT == POSITIONS == 1: 5895_0746879266_03_WRI-L2_M008
SCENARIO 1: COUNT == POSITIONS == 1: 1464_0490421777_01_WRI-R2_M011
SCENARIO 4: COUNT < POSITIONS: 5746_0258880684_02_WRI-L1_M010
	 SCENARIO 4.1: 5746_0258880684_02_WRI-L1_M010


  0%|                                        | 13/13995 [00:00<10:51, 21.45it/s]

...

100%|████████████████████████████████████▉| 13987/13995 [09:04<00:00, 22.03it/s]

SCENARIO 3: COUNT == POSITIONS == 2: 4299_0639389095_03_WRI-L1_M012
	 SCENARIO 3.1 AP: 4299_0639389095_03_WRI-L1_M012
SCENARIO 3: COUNT == POSITIONS == 2: 1599_1069129302_03_WRI-R1_M013
	 SCENARIO 3.1 AP: 1599_1069129302_03_WRI-R1_M013
SCENARIO 1: COUNT == POSITIONS == 1: 1621_0411055747_01_WRI-R1_M015
SCENARIO 3: COUNT == POSITIONS == 2: 2937_1139613198_01_WRI-R1_M016
	 SCENARIO 3.1 AP: 2937_1139613198_01_WRI-R1_M016


100%|████████████████████████████████████▉| 13993/13995 [09:04<00:00, 21.21it/s]

SCENARIO 2: COUNT > POSITIONS: 0458_1054495360_01_WRI-R2_M008
ELLO
SCENARIO 1: COUNT == POSITIONS == 1: 5837_0367496010_02_WRI-L2_F011
SCENARIO 2: COUNT > POSITIONS: 3889_0922875184_02_WRI-R2_M009
ELLO
SCENARIO 4: COUNT < POSITIONS: 2070_0831290055_03_WRI-L1_F012
	 SCENARIO 4.1: 2070_0831290055_03_WRI-L1_F012
SCENARIO 1: COUNT == POSITIONS == 1: 5054_1148719074_01_WRI-R1_M010
SCENARIO 1: COUNT == POSITIONS == 1: 3844_0191528370_02_WRI-L2_M006
SCENARIO 1.15: ERROR
ELLO
SCENARIO 3: COUNT == POSITIONS == 2: 0566_1179399571_04_WRI-R1_M015
	 SCENARIO 3.1 AP: 0566_1179399571_04_WRI-R1_M015


SCENARIO 2: COUNT > POSITIONS: 1263_0954115781_03_WRI-R1_F012
ELLO


In [ ]:
_main_output_dir = OrderedDict(sorted(_main_output_dir.items(), key=lambda x: x[1], reverse=True))
_main_output_dir

In [ ]:
# COUNT EVERYTHING!!!! :)
# Function that extract data
_diagnoses_list, _, _ = obtain_data(_ann_json_dir_filtered)

# Obtain dictionary
_diagnoses_dict = build_dictionary_from_list(_diagnoses_list)
_diagnoses_dict = OrderedDict(sorted(_diagnoses_dict.items(), key=lambda x: x[1], reverse=True))
print(_diagnoses_dict)


In [ ]:
_index = retrive_fracture_index_from_case_name(_output_list, _name_list2[-1])
print(_output_list[_index][1:])
display_fractures(_output_list[_index])
_scenarion, _name = check_scenarios(_output_list[_index], "/home/franko/Desktop/Frlan/Novo/Scripts/Results")